In [4]:
!pip install transformers
!pip install mosestokenizer
!pip install sentencepiece

In [2]:
from transformers import MarianMTModel, MarianTokenizer

#START: COPIED FROM https://amitness.com/back-translation/"
def translate(texts, model, tokenizer, language):
  """Translate to target language"""
  # Format the text as expected by the model
  formatter_fn = lambda txt: f"{txt}" if language == "en" else f">>{language}<< {txt}"
  original_texts = [formatter_fn(txt) for txt in texts]
  # Tokenize text
  tokens = tokenizer.prepare_seq2seq_batch(original_texts, return_tensors="pt")
  # Translate
  translated = model.generate(**tokens)
  # Decode, convert tokens to text
  translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return translated_texts

#END: COPIED FROM https://amitness.com/back-translation/"

def back_translate(texts):
  # model for English -> Romance
  tmp_model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
  mid_tokenizer = MarianTokenizer.from_pretrained(tmp_model_name)
  mid_model = MarianMTModel.from_pretrained(tmp_model_name)

  # model for Romance -> English
  src_model_name = 'Helsinki-NLP/opus-mt-ROMANCE-en'
  src_tokenizer = MarianTokenizer.from_pretrained(src_model_name)
  src_model = MarianMTModel.from_pretrained(src_model_name)

  translated_text = translate(texts, mid_model, mid_tokenizer, "es")
  print("translated")
  print(translated_text)
  back_translated = translate(translated_text, src_model, src_tokenizer, "en")

  return back_translated

In [ ]:
import json

with open("/Users/angela/Documents/CovidET-main/data_withposts/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS_additional_emotions.json") as f:
    dataset = json.load(f)

ind_to_post_info = {}

emotion_set = {'joy', 'trust', 'disgust', 'confusion', 'resilience'}
size = len(dataset)
count = 0
iter = 0
for k in dataset:
  annotations = {}
  for j, annotation in enumerate(dataset[k]['Annotations']): #annotation 0, annotation 1
    emotions = []
    for hit in dataset[k]['Annotations'][annotation]:  #emotion, abstractive pair
        if hit['Emotion'] in emotion_set:
          emotions.append({"Emotion":hit['Emotion'], "Abstractive": hit["Abstractive"], "Reddit Post Original": dataset[k]['Reddit Post']})
    if len(emotions) > 0:
      annotations["Annotation " + str(j)] = emotions
  if len(annotations) > 0: # if dataset[k] contains targeted emotion
    ind_to_post_info[str(size + count)] = {}
    ind_to_post_info[str(size + count)]["Annotations"] = annotations
    ind_to_post_info[str(size + count)]["Reddit Post"] = back_translate([dataset[k]['Reddit Post']])
    count += 1 # total number of new posts added so far

In [6]:
json_object = json.dumps(ind_to_post_info, indent = 2)
with open("BT_only_new.json", "w") as outfile:
    outfile.write(json_object)
print(json_object)

{
  "1200": {
    "Annotations": {
      "Annotation 0": [
        {
          "Emotion": "confusion",
          "Abstractive": "Now Im seeing so much conflicting information at the same time and Im confused",
          "Reddit Post Original": "Im suppose to get my first vaccination soon. Everyone else in my household and my mother is fully vaccinated. After doing a lot of research to questions I had regarding the vaccines I felt more comfortable finally booking to get one. It seemed like they are/were working. Lockdowns were being lifted, I saw online people doing things social and doctors celebrating etc. I avoid the news about covid for my mental health. Pretty much what I find out about the pandemic pops up on my feed. Now Im seeing so much conflicting information at the same time and Im confused and defeated. UK says pfizer vaccine is 88% effective against delta variant but is that only against hospitalizations? What about actually getting it and suffering? Israel says pfizer went